In [17]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings("ignore")
import os

In [26]:
paths=[]
labels=[]
for dirname, _, filenames in os.walk(r'C:\Users\rachi\Desktop\py\Machine\CodersCave_internship\TESS Toronto emotional speech set data'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label=filename.split('_')[-1]
        label=label.split('.')[0]
        labels.append(label.lower())

In [27]:
df=pd.DataFrame()
df['speech']=paths
df['label']=labels

In [28]:
df['label']

0       angry
1       angry
2       angry
3       angry
4       angry
        ...  
5469      sad
5470      sad
5471      sad
5472      sad
5473      sad
Name: label, Length: 5474, dtype: object

In [29]:
def extract_mfcc(filename):
    y,sr=librosa.load(filename,duration=3,offset=0.5)
    mfcc=np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)
    return mfcc

In [30]:
x_mfcc=df['speech'].apply(lambda X: extract_mfcc(X))

In [37]:
x=[X for X in x_mfcc]
x=np.array(x)
x.shape

(5474, 40)

In [40]:
X=np.expand_dims(x,-1)
X.shape

(5474, 40, 1)

In [54]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
y=enc.fit_transform(df[['label']])
y=y.toarray()
y.shape

(5474, 7)

In [55]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

In [60]:
Model=Sequential([
    LSTM(123,return_sequences= False, input_shape=(40,1)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32,activation='relu'),
    Dropout(0.2),
    Dense(7,activation='softmax')
])
Model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
Model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 123)               61500     
                                                                 
 dense_9 (Dense)             (None, 64)                7936      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 7)                 231       
                                                                 
Total params: 71,747
Trainable params: 71,747
Non-trai

In [61]:
h=Model.fit(x,y,validation_split=0.2,epochs=10,batch_size=512,shuffle=True)

Epoch 1/10
9/9 [==============================] - 5s 311ms/step - loss: 1.9028 - accuracy: 0.2797 - val_loss: 1.8500 - val_accuracy: 0.3489
Epoch 2/10
9/9 [==============================] - 2s 245ms/step - loss: 1.6547 - accuracy: 0.4421 - val_loss: 1.5138 - val_accuracy: 0.4904
Epoch 3/10
9/9 [==============================] - 2s 262ms/step - loss: 1.3474 - accuracy: 0.5118 - val_loss: 1.1196 - val_accuracy: 0.5918
Epoch 4/10
9/9 [==============================] - 2s 252ms/step - loss: 1.1380 - accuracy: 0.5675 - val_loss: 0.9126 - val_accuracy: 0.6155
Epoch 5/10
9/9 [==============================] - 2s 240ms/step - loss: 0.9622 - accuracy: 0.6422 - val_loss: 0.6998 - val_accuracy: 0.7151
Epoch 6/10
9/9 [==============================] - 2s 213ms/step - loss: 0.7707 - accuracy: 0.7344 - val_loss: 0.4868 - val_accuracy: 0.8639
Epoch 7/10
9/9 [==============================] - 2s 207ms/step - loss: 0.5571 - accuracy: 0.8130 - val_loss: 0.2634 - val_accuracy: 0.9452
Epoch 8/10
9/9 [====